<a href="https://colab.research.google.com/github/marceloperrone/quind24/blob/main/Step_2_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [28]:
import pandas as pd

meta_info = {}
meta_info['df_master_name'] = 'V2.3-20240709T133822.csv'
df_master = pd.read_csv('/content/drive/MyDrive/FCB/Rural Reframe/Datasets/Exports/Base/' + meta_info['df_master_name'],
                        dtype={'Geo_FIPS': str,
                               'State_FIPS': str,
                               'County_FIPS': str,
                               'COUNTY_NAME': str,
                               'STATE_NAME': str,
                               'R_CONTINUUM_CODE': str,
                               'ECON_TYPE_CAT':str,
                               'ECON_TYPE_RETDEST_BIN':str})

df_master.drop('Unnamed: 0', axis=1, inplace=True)

main_features = ['TOT_POP_NUM','NHWHITE_PCT','NHBLACK_PCT','HISPANIC_PCT','ECON_TYPE_CAT','ECON_TYPE_RETDEST_BIN','JOBS_2022_NUM','FARM_PERSINC_1992_PCT','FARM_PERSINC_2002_PCT','FARM_PERSINC_2012_PCT','FARM_PERSINC_2022_PCT','UNEMP_RATE_2000_PCT','UNEMP_RATE_2005_PCT','UNEMP_RATE_2010_PCT','UNEMP_RATE_2015_PCT','UNEMP_RATE_2020_PCT','UNEMP_RATE_2021_PCT','UNEMP_RATE_2022_PCT','EDU_NOHIGH_2021_PCT','EDU_HIGHONLY_2021_PCT','EDU_COLLEGE_2021_PCT','EDU_BACH_2021_PCT','ELE_DEMOCRAT_2020_PCT','ELE_REPUBLICAN_2020_PCT','POV_POVERTYPOP_2021_NUM','FOOD_INSRATE_2021_PCT','CVD19_PRIMARY_PCT','CVD19_BOOSTER_PCT','DISABILITY_2021_PCT','DEPRESSION_2021_PCT','DIABETES_2021_PCT','OBESITY_2021_PCT','GHLTH_2021_PCT','MHLTH_2021_PCT','PHLTH_2021_PCT','ACCESS2_2021_PCT','BINGE_2021_PCT','CSMOKING_2021_PCT','PCH_R_BIRTH_2022_PCT','PCH_R_DEATH_2022_PCT','PCH_R_NATURALCHG_2021_PCT','PCH_R_NATURALCHG_2022_PCT','PCH_R_NETMIG_2021_PCT','PCH_R_NETMIG_2022_PCT','GDP_2022_USD','GDP_GROWTH_2020_PCT','GDP_GROWTH_2021_PCT','GDP_GROWTH_2022_PCT','GINI_2022_IND','IOC_O_MAN_22_PCT','IOC_O_SER_22_PCT','IOC_O_SAO_22_PCT','IOC_O_NAT_22_PCT','IOC_O_PRO_22_PCT','SSC_UNM_WOMEN_BIRTHP12M_PCT','SSC_SAME_HOUSE_YA_PCT','SSC_ENG_NOT_WELL_PCT','SSC_HH_COMPUTER_PCT','SSC_HH_BROADBAND_PCT','SSC_MARITAL_15PLUS_NUM','SSC_MARITAL_NVM_PCT','SSC_MARITAL_MARRIED_PCT','SSC_MARITAL_SEPARATED_PCT','SSC_MARITAL_WIDOWED_PCT','SSC_MARITAL_DIVORCED_PCT','HINS_U19YRS_NUM','HINS_1934YRS_NUM','HINS_3564YRS_NUM','HINS_65UPYRS_NUM','HINS_NOTINSURED_PCT','HINS_NOTINSURED_19U_PCT','HINS_NOTINSURED_1934YRS_PCT','HINS_NOTINSURED_3564YRS_PCT','HINS_NOTINSURED_65UPYRS_PCT','NI_INS_PRI_PCT','NI_INS_PUB_PCT','HCPWF_PHYSPER1000_PCT','HCPWF_NURSESPER1000_PCT','HCPWF_DENTSPER1000_PCT','DRUG_MORTALITY_RATE_PCT']

df_main_features = df_master[main_features]

meta_info['main_features'] = list(main_features)

In [29]:
from sklearn.feature_selection import VarianceThreshold

vt_threshold = 0.002

selector = VarianceThreshold(threshold=vt_threshold)
df_vt = selector.fit_transform(df_main_features)

# Get the boolean mask indicating which features are kept
mask = selector.get_support()

# List of all original features
original_features = df_main_features.columns
kept_features = original_features[mask]
dropped_features = original_features[~mask]


df_vt = df_main_features.drop(columns=dropped_features)


meta_info['vt_threshold'] = vt_threshold
meta_info['variance_threshold_round1_dropped_features'] = list(dropped_features)

print(len(meta_info['variance_threshold_round1_dropped_features']))
print(meta_info['variance_threshold_round1_dropped_features'])

25
['UNEMP_RATE_2000_PCT', 'UNEMP_RATE_2005_PCT', 'UNEMP_RATE_2010_PCT', 'UNEMP_RATE_2015_PCT', 'UNEMP_RATE_2020_PCT', 'UNEMP_RATE_2021_PCT', 'UNEMP_RATE_2022_PCT', 'FOOD_INSRATE_2021_PCT', 'DEPRESSION_2021_PCT', 'DIABETES_2021_PCT', 'MHLTH_2021_PCT', 'PHLTH_2021_PCT', 'BINGE_2021_PCT', 'CSMOKING_2021_PCT', 'PCH_R_BIRTH_2022_PCT', 'PCH_R_DEATH_2022_PCT', 'GINI_2022_IND', 'IOC_O_SER_22_PCT', 'IOC_O_SAO_22_PCT', 'IOC_O_NAT_22_PCT', 'SSC_SAME_HOUSE_YA_PCT', 'SSC_MARITAL_SEPARATED_PCT', 'SSC_MARITAL_WIDOWED_PCT', 'SSC_MARITAL_DIVORCED_PCT', 'HINS_NOTINSURED_65UPYRS_PCT']


In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, MinMaxScaler

# Step 1: Impute missing values
imputer = SimpleImputer(strategy='mean')
df_imputed = pd.DataFrame(imputer.fit_transform(df_vt.select_dtypes(include=['float64', 'int64'])),
                          columns=df_vt.select_dtypes(include=['float64', 'int64']).columns)

# Include the categorical features back
categorical_features = df_vt.select_dtypes(include=['object']).columns
df_imputed[categorical_features] = df_vt[categorical_features]

# Step 2: Separate features by type
percentage_features = [col for col in df_imputed.columns if col.endswith('_PCT')]
non_percentage_features = [col for col in df_imputed.select_dtypes(include=['float64', 'int64']).columns if col not in percentage_features]

# Print the separated feature lists for debugging
print("Percentage features:", percentage_features)
print("Non-percentage numeric features:", non_percentage_features)
print("Categorical features:", categorical_features)

# Step 3: Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num_non_pct', RobustScaler(), non_percentage_features),
        ('num_pct', MinMaxScaler(), percentage_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ]
)

# Apply scaling
df_transformed = preprocessor.fit_transform(df_imputed)


# Get the feature names for the final dataframe
# For OneHotEncoder, we get the actual feature names from the encoder
ohe = preprocessor.named_transformers_['cat']
cat_feature_names = ohe.get_feature_names_out(categorical_features)

# Combine feature names
all_feature_names = non_percentage_features + percentage_features + list(cat_feature_names)

# Convert the transformed data back to a dataframe
df_transformed = pd.DataFrame(df_transformed, columns=all_feature_names)

meta_info['preprocessor_round1'] = print(preprocessor)

Percentage features: ['NHWHITE_PCT', 'NHBLACK_PCT', 'HISPANIC_PCT', 'FARM_PERSINC_1992_PCT', 'FARM_PERSINC_2002_PCT', 'FARM_PERSINC_2012_PCT', 'FARM_PERSINC_2022_PCT', 'EDU_NOHIGH_2021_PCT', 'EDU_HIGHONLY_2021_PCT', 'EDU_COLLEGE_2021_PCT', 'EDU_BACH_2021_PCT', 'ELE_DEMOCRAT_2020_PCT', 'ELE_REPUBLICAN_2020_PCT', 'CVD19_PRIMARY_PCT', 'CVD19_BOOSTER_PCT', 'DISABILITY_2021_PCT', 'OBESITY_2021_PCT', 'GHLTH_2021_PCT', 'ACCESS2_2021_PCT', 'PCH_R_NATURALCHG_2021_PCT', 'PCH_R_NATURALCHG_2022_PCT', 'PCH_R_NETMIG_2021_PCT', 'PCH_R_NETMIG_2022_PCT', 'GDP_GROWTH_2020_PCT', 'GDP_GROWTH_2021_PCT', 'GDP_GROWTH_2022_PCT', 'IOC_O_MAN_22_PCT', 'IOC_O_PRO_22_PCT', 'SSC_UNM_WOMEN_BIRTHP12M_PCT', 'SSC_ENG_NOT_WELL_PCT', 'SSC_HH_COMPUTER_PCT', 'SSC_HH_BROADBAND_PCT', 'SSC_MARITAL_NVM_PCT', 'SSC_MARITAL_MARRIED_PCT', 'HINS_NOTINSURED_PCT', 'HINS_NOTINSURED_19U_PCT', 'HINS_NOTINSURED_1934YRS_PCT', 'HINS_NOTINSURED_3564YRS_PCT', 'NI_INS_PRI_PCT', 'NI_INS_PUB_PCT', 'HCPWF_PHYSPER1000_PCT', 'HCPWF_NURSESPER1000_P

In [31]:
import numpy as np

# Compute correlation matrix
corr_matrix = df_transformed.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]


# Drop features
df_transformed_cm = df_transformed.drop(columns=to_drop)


kept_features_cm = list(df_transformed_cm.columns)

meta_info['correl_matrix_kept_round1'] = kept_features_cm
meta_info['correl_matrix_dropped_round1'] = list(to_drop)


print(len(meta_info['correl_matrix_dropped_round1']))
print(meta_info['correl_matrix_dropped_round1'])
print(meta_info['correl_matrix_kept_round1'])


15
['JOBS_2022_NUM', 'POV_POVERTYPOP_2021_NUM', 'GDP_2022_USD', 'SSC_MARITAL_15PLUS_NUM', 'HINS_U19YRS_NUM', 'HINS_1934YRS_NUM', 'HINS_3564YRS_NUM', 'HINS_65UPYRS_NUM', 'ELE_REPUBLICAN_2020_PCT', 'GHLTH_2021_PCT', 'PCH_R_NATURALCHG_2022_PCT', 'SSC_HH_BROADBAND_PCT', 'HINS_NOTINSURED_1934YRS_PCT', 'HINS_NOTINSURED_3564YRS_PCT', 'ECON_TYPE_RETDEST_BIN_nan']
['TOT_POP_NUM', 'NHWHITE_PCT', 'NHBLACK_PCT', 'HISPANIC_PCT', 'FARM_PERSINC_1992_PCT', 'FARM_PERSINC_2002_PCT', 'FARM_PERSINC_2012_PCT', 'FARM_PERSINC_2022_PCT', 'EDU_NOHIGH_2021_PCT', 'EDU_HIGHONLY_2021_PCT', 'EDU_COLLEGE_2021_PCT', 'EDU_BACH_2021_PCT', 'ELE_DEMOCRAT_2020_PCT', 'CVD19_PRIMARY_PCT', 'CVD19_BOOSTER_PCT', 'DISABILITY_2021_PCT', 'OBESITY_2021_PCT', 'ACCESS2_2021_PCT', 'PCH_R_NATURALCHG_2021_PCT', 'PCH_R_NETMIG_2021_PCT', 'PCH_R_NETMIG_2022_PCT', 'GDP_GROWTH_2020_PCT', 'GDP_GROWTH_2021_PCT', 'GDP_GROWTH_2022_PCT', 'IOC_O_MAN_22_PCT', 'IOC_O_PRO_22_PCT', 'SSC_UNM_WOMEN_BIRTHP12M_PCT', 'SSC_ENG_NOT_WELL_PCT', 'SSC_HH_COMPUT

In [32]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

meta_info['PCA_components'] = 0.95
meta_info['n_clusters'] = 4

pca = PCA(n_components=meta_info['PCA_components'])
df_pca = pca.fit_transform(df_transformed_cm)
kmeans_pca = KMeans(n_clusters=meta_info['n_clusters'], random_state=1984, init='k-means++', n_init=20)

clusters_pca_kmeans = kmeans_pca.fit_predict(df_pca)

meta_info['silhouette_score'] = silhouette_score(df_transformed_cm, clusters_pca_kmeans).round(2)
meta_info['silhouette_score_R_CONTINUUM'] = silhouette_score(df_transformed_cm, df_master['R_CONTINUUM_CODE']).round(2)

print(meta_info['silhouette_score'])
print(meta_info['silhouette_score_R_CONTINUUM'])


data = df_master.copy()
data.loc[:,'cluster1'] = clusters_pca_kmeans
data['cluster1'] = data['cluster1'].astype(str)

0.8
-0.15


In [33]:
TOT_POP_NUM = data['TOT_POP_NUM'].sum()
GDP_2022_USD = data['GDP_2022_USD'].sum()

In [34]:
def cluster_breakdown(data,groupby_list):
    r = data.groupby(groupby_list).agg({'GDP_2022_USD': 'sum', 'Geo_FIPS':'count',
                                        'FARM_PERSINC_2022_PCT':'mean','IOC_I_AGR_22_PCT':'mean','TOT_POP_NUM': 'sum'})
    r = r.sort_values(by=groupby_list[0])
    r['TOT_POP_PCT'] = (r['TOT_POP_NUM']/TOT_POP_NUM).apply(lambda x: "{:.2%}".format(x))
    r['FARM_PERSINC_2022_PCT'] = r['FARM_PERSINC_2022_PCT'].apply(lambda x: "{:.2%}".format(x))
    r['IOC_I_AGR_22_PCT'] = r['IOC_I_AGR_22_PCT'].apply(lambda x: "{:.2%}".format(x))
    r['TOT_POP_NUM'] = r['TOT_POP_NUM'].astype(int)
    r['TOT_POP_NUM'] = r['TOT_POP_NUM'].apply(lambda x: "{:,}".format(x))
    r['GDP_2022_USD'] = (r['GDP_2022_USD']/GDP_2022_USD).round(4).apply(lambda x: "{:.1%}".format(x))
    return r

In [35]:
cluster_breakdown(data,['cluster1'])

,GDP_2022_USD,Geo_FIPS,FARM_PERSINC_2022_PCT,IOC_I_AGR_22_PCT,TOT_POP_NUM,TOT_POP_PCT
cluster1,,,,,,
0,18.1%,17,0.06%,0.68%,"48,301,771",14.49%
1,33.7%,2963,3.83%,6.49%,"150,693,822",45.21%
2,3.8%,1,0.03%,0.46%,"9,721,138",2.92%
3,44.4%,163,0.23%,1.11%,"124,570,826",37.38%


In [36]:
cluster_breakdown(data,['cluster1','R_CONTINUUM_CODE'])

GDP_2022_USD  Geo_FIPS FARM_PERSINC_2022_PCT  \
cluster1 R_CONTINUUM_CODE                                                
0        1                       18.1%        17                 0.06%   
1        1                        7.7%       310                 0.57%   
         2                       10.0%       337                 1.40%   
         3                        6.7%       358                 1.83%   
         4                        2.7%       201                 1.27%   
         5                        1.0%        76                 1.62%   
         6                        2.2%       379                 3.28%   
         7                        1.3%       247                 3.16%   
         8                        1.1%       466                 5.25%   
         9                        1.0%       589                 8.64%   
2        1                        3.8%         1                 0.03%   
3        1                       37.8%       115                 0.07%   
         2                        6.6%        48                 0.64%   

                          IOC_I_AGR_22_PCT TOT_POP_NUM TOT_POP_PCT  
cluster1 R_CONTINUUM_CODE                                           
0        1                           0.68%  48,301,771      14.49%  
1        1                           1.83%  34,070,149      10.22%  
         2                           2.88%  41,190,067      12.36%  
         3                           3.73%  29,532,447       8.86%  
         4                           3.05%  13,442,092       4.03%  
         5                           5.47%   4,169,503       1.25%  
         6                           5.29%  11,543,934       3.46%  
         7                           6.32%   6,025,585       1.81%  
         8                           8.49%   6,185,109       1.86%  
         9                          13.27%   4,534,936       1.36%  
2        1                           0.46%   9,721,138       2.92%  
3        1                           0.67%  97,330,681      29.20%  
         2                           2.16%  27,240,145       8.17%

In [37]:
data['cluster2'] = 'U'
data_round2 = data[data['cluster1'].isin(['1'])]

In [38]:
data_round2_mf = data_round2[main_features]

vt_selector_r2 = VarianceThreshold(threshold=0.05)
vt_obj_2 = vt_selector_r2.fit_transform(data_round2_mf)

vtmask_r2 = vt_selector_r2.get_support()
original_features_r2 = data_round2_mf.columns
kept_features_r2 = original_features_r2[vtmask_r2]
dropped_features_r2 = original_features_r2[~vtmask_r2]

df_vt_round2 = data_round2_mf.drop(columns=dropped_features_r2)

meta_info['variance_threshold_round2_dropped_features'] = list(dropped_features_r2)
meta_info['variance_threshold_round2_kept_features'] = list(df_vt_round2.columns)

print("DROPPED")
print(len(meta_info['variance_threshold_round2_dropped_features']))
print(meta_info['variance_threshold_round2_dropped_features'])
print("KEPT")
print(len(meta_info['variance_threshold_round2_kept_features']))
print(meta_info['variance_threshold_round2_kept_features'])

DROPPED
63
['NHWHITE_PCT', 'NHBLACK_PCT', 'HISPANIC_PCT', 'FARM_PERSINC_1992_PCT', 'FARM_PERSINC_2002_PCT', 'FARM_PERSINC_2012_PCT', 'FARM_PERSINC_2022_PCT', 'UNEMP_RATE_2000_PCT', 'UNEMP_RATE_2005_PCT', 'UNEMP_RATE_2010_PCT', 'UNEMP_RATE_2015_PCT', 'UNEMP_RATE_2020_PCT', 'UNEMP_RATE_2021_PCT', 'UNEMP_RATE_2022_PCT', 'EDU_NOHIGH_2021_PCT', 'EDU_HIGHONLY_2021_PCT', 'EDU_COLLEGE_2021_PCT', 'EDU_BACH_2021_PCT', 'ELE_DEMOCRAT_2020_PCT', 'ELE_REPUBLICAN_2020_PCT', 'FOOD_INSRATE_2021_PCT', 'CVD19_PRIMARY_PCT', 'CVD19_BOOSTER_PCT', 'DISABILITY_2021_PCT', 'DEPRESSION_2021_PCT', 'DIABETES_2021_PCT', 'OBESITY_2021_PCT', 'GHLTH_2021_PCT', 'MHLTH_2021_PCT', 'PHLTH_2021_PCT', 'ACCESS2_2021_PCT', 'BINGE_2021_PCT', 'CSMOKING_2021_PCT', 'PCH_R_BIRTH_2022_PCT', 'PCH_R_DEATH_2022_PCT', 'PCH_R_NATURALCHG_2021_PCT', 'PCH_R_NATURALCHG_2022_PCT', 'PCH_R_NETMIG_2021_PCT', 'PCH_R_NETMIG_2022_PCT', 'GINI_2022_IND', 'IOC_O_MAN_22_PCT', 'IOC_O_SER_22_PCT', 'IOC_O_SAO_22_PCT', 'IOC_O_NAT_22_PCT', 'IOC_O_PRO_22_PC

In [39]:

# Step 1: Impute missing values
imputer_round2 = SimpleImputer(strategy='mean')
df_imputed_round2 = pd.DataFrame(imputer_round2.fit_transform(df_vt_round2.select_dtypes(include=['float64', 'int64'])), columns=df_vt_round2.select_dtypes(include=['float64', 'int64']).columns)

# Include the categorical features back
categorical_features_round2 = df_vt_round2.select_dtypes(include=['object']).columns
df_imputed_round2[categorical_features_round2] = df_vt_round2[categorical_features_round2]

# Step 2: Separate features by type
percentage_features_round2 = [col for col in df_imputed_round2.columns if col.endswith('_PCT')]
non_percentage_features_round2 = [col for col in df_imputed_round2.select_dtypes(include=['float64', 'int64']).columns if col not in percentage_features_round2]

# Print the separated feature lists for debugging
print("Percentage features:", percentage_features_round2)
print("Non-percentage numeric features:", non_percentage_features_round2)
print("Categorical features:", categorical_features_round2)

# Step 3: Define the preprocessor
preprocessor_round2 = ColumnTransformer(
    transformers=[
        ('num_non_pct', RobustScaler(), non_percentage_features_round2),
        ('num_pct', MinMaxScaler(), percentage_features_round2),
        ('cat', OneHotEncoder(drop='first'), categorical_features_round2)
    ]
)

# Apply scaling
df_transformed_round2 = preprocessor_round2.fit_transform(df_imputed_round2)


# Get the feature names for the final dataframe
# For OneHotEncoder, we get the actual feature names from the encoder
ohe_round2 = preprocessor_round2.named_transformers_['cat']
cat_feature_names_round2 = ohe_round2.get_feature_names_out(categorical_features_round2)

# Combine feature names
all_feature_names_round2 = non_percentage_features_round2 + percentage_features_round2 + list(cat_feature_names_round2)

# Convert the transformed data back to a dataframe
df_transformed_round2 = pd.DataFrame(df_transformed_round2, columns=all_feature_names_round2)




Percentage features: ['GDP_GROWTH_2020_PCT', 'GDP_GROWTH_2021_PCT', 'GDP_GROWTH_2022_PCT', 'HCPWF_PHYSPER1000_PCT', 'HCPWF_NURSESPER1000_PCT', 'HCPWF_DENTSPER1000_PCT']
Non-percentage numeric features: ['TOT_POP_NUM', 'JOBS_2022_NUM', 'POV_POVERTYPOP_2021_NUM', 'GDP_2022_USD', 'SSC_MARITAL_15PLUS_NUM', 'HINS_U19YRS_NUM', 'HINS_1934YRS_NUM', 'HINS_3564YRS_NUM', 'HINS_65UPYRS_NUM']
Categorical features: Index(['ECON_TYPE_CAT', 'ECON_TYPE_RETDEST_BIN'], dtype='object')


In [40]:
# Compute correlation matrix
corr_matrix_round2 = df_transformed_round2.corr().abs()
upper_round2 = corr_matrix_round2.where(np.triu(np.ones(corr_matrix_round2.shape), k=1).astype(bool))
to_drop_round2 = [column for column in upper_round2.columns if any(upper_round2[column] > 0.85)]


# Drop features
df_transformed_cm_round2 = df_transformed_round2.drop(columns=to_drop_round2)


meta_info['correl_matrix_dropped_round2'] = list(to_drop_round2)
meta_info['correl_matrix_kept_round2'] = list(df_transformed_cm_round2.columns)

print("DROPPED")
print(len(meta_info['correl_matrix_dropped_round2']))
print(meta_info['correl_matrix_dropped_round2'])

print("KEPT")
print(len(meta_info['correl_matrix_kept_round2']))
print(meta_info['correl_matrix_kept_round2'])



DROPPED
9
['JOBS_2022_NUM', 'POV_POVERTYPOP_2021_NUM', 'GDP_2022_USD', 'SSC_MARITAL_15PLUS_NUM', 'HINS_U19YRS_NUM', 'HINS_1934YRS_NUM', 'HINS_3564YRS_NUM', 'HINS_65UPYRS_NUM', 'ECON_TYPE_RETDEST_BIN_nan']
KEPT
14
['TOT_POP_NUM', 'GDP_GROWTH_2020_PCT', 'GDP_GROWTH_2021_PCT', 'GDP_GROWTH_2022_PCT', 'HCPWF_PHYSPER1000_PCT', 'HCPWF_NURSESPER1000_PCT', 'HCPWF_DENTSPER1000_PCT', 'ECON_TYPE_CAT_1.0', 'ECON_TYPE_CAT_2.0', 'ECON_TYPE_CAT_3.0', 'ECON_TYPE_CAT_4.0', 'ECON_TYPE_CAT_5.0', 'ECON_TYPE_CAT_nan', 'ECON_TYPE_RETDEST_BIN_1.0']


In [41]:
meta_info['PCA_components_r2'] = 0.95
meta_info['n_clusters_r2'] = 4

pca_round2 = PCA(n_components=meta_info['PCA_components_r2'])
df_pca_round2 = pca_round2.fit_transform(df_transformed_cm_round2)
kmeans_pca_round2 = KMeans(n_clusters=meta_info['n_clusters_r2'], random_state=1984, init='k-means++', n_init=20)

clusters_pca_kmeans_round2 = kmeans_pca.fit_predict(df_pca_round2)


meta_info['silhouette_score_round2'] = silhouette_score(df_transformed_cm_round2, clusters_pca_kmeans_round2).round(2)
print(meta_info['silhouette_score_round2'])
data_round2.loc[:,'cluster2'] = clusters_pca_kmeans_round2
data_round2.loc[:,'cluster2'] = data_round2['cluster2'].astype(str)

0.29


In [42]:
cluster_breakdown(data_round2,['cluster2'])

,GDP_2022_USD,Geo_FIPS,FARM_PERSINC_2022_PCT,IOC_I_AGR_22_PCT,TOT_POP_NUM,TOT_POP_PCT
cluster2,,,,,,
0,7.0%,2065,5.17%,8.28%,"35,338,435",10.60%
1,9.2%,230,0.42%,1.68%,"39,463,017",11.84%
2,9.5%,113,0.33%,1.36%,"35,246,184",10.58%
3,8.1%,555,0.96%,2.87%,"40,646,186",12.20%


In [43]:
data_round2 = data_round2[['Geo_FIPS','cluster2']]
data_all = pd.merge(data, data_round2, on='Geo_FIPS', how='left')
data_all.rename(columns={'cluster2_x': 'cluster2'}, inplace=True)
data_all.loc[data_all['cluster2_y'].notnull(), 'cluster2'] = data_all['cluster2_y']
data_all = data_all.drop(columns=['cluster2_y'])
cluster_breakdown(data_all,['cluster1','cluster2'])

GDP_2022_USD  Geo_FIPS FARM_PERSINC_2022_PCT  \
cluster1 cluster2                                                
0        U               18.1%        17                 0.06%   
1        0                7.0%      2065                 5.17%   
         1                9.2%       230                 0.42%   
         2                9.5%       113                 0.33%   
         3                8.1%       555                 0.96%   
2        U                3.8%         1                 0.03%   
3        U               44.4%       163                 0.23%   

                  IOC_I_AGR_22_PCT  TOT_POP_NUM TOT_POP_PCT  
cluster1 cluster2                                            
0        U                   0.68%   48,301,771      14.49%  
1        0                   8.28%   35,338,435      10.60%  
         1                   1.68%   39,463,017      11.84%  
         2                   1.36%   35,246,184      10.58%  
         3                   2.87%   40,646,186      12.20%  
2        U                   0.46%    9,721,138       2.92%  
3        U                   1.11%  124,570,826      37.38%

In [44]:
data_all['labels'] = 'A'
data_all.loc[(data_all['cluster1'] == '0') & (data_all['cluster2'] == 'U'), 'labels'] = 'Urban 1'
data_all.loc[(data_all['cluster1'] == '2') & (data_all['cluster2'] == 'U'), 'labels'] = 'Urban 2'
data_all.loc[(data_all['cluster1'] == '3') & (data_all['cluster2'] == 'U'), 'labels'] = 'Urban 3'
data_all.loc[(data_all['cluster1'] == '1') & (data_all['cluster2'] == '0'), 'labels'] = 'Rural Reframe 1'
data_all.loc[(data_all['cluster1'] == '1') & (data_all['cluster2'] == '1'), 'labels'] = 'Rural Reframe 2'
data_all.loc[(data_all['cluster1'] == '1') & (data_all['cluster2'] == '2'), 'labels'] = 'Rural Reframe 3'
data_all.loc[(data_all['cluster1'] == '1') & (data_all['cluster2'] == '3'), 'labels'] = 'Rural Reframe 4'

In [45]:
df_clustered_selected = data_all[main_features]
df_clustered_selected = df_clustered_selected.replace(['(NA)', 'N/A'], np.nan)
df_clustered_selected = df_clustered_selected[df_clustered_selected.select_dtypes(include=['int64', 'float64']).columns].apply(lambda x: x.fillna(x.mean()), axis=0)

meta_info['silhouette_score_round3'] = silhouette_score(df_clustered_selected, data_all['labels']).round(2)
meta_info['silhouette_score_R_CONTINUUM_CODE_round3'] = silhouette_score(df_clustered_selected, data_all['R_CONTINUUM_CODE']).round(2)

print('LABELS: '+ str(meta_info['silhouette_score_round3']) + " / CONTINUUM CODE: " + str(meta_info['silhouette_score_R_CONTINUUM_CODE_round3'])+'\n')



LABELS: 0.44 / CONTINUUM CODE: -0.26



In [46]:
cluster_breakdown(data_all,['labels'])

,GDP_2022_USD,Geo_FIPS,FARM_PERSINC_2022_PCT,IOC_I_AGR_22_PCT,TOT_POP_NUM,TOT_POP_PCT
labels,,,,,,
Rural Reframe 1,7.0%,2065,5.17%,8.28%,"35,338,435",10.60%
Rural Reframe 2,9.2%,230,0.42%,1.68%,"39,463,017",11.84%
Rural Reframe 3,9.5%,113,0.33%,1.36%,"35,246,184",10.58%
Rural Reframe 4,8.1%,555,0.96%,2.87%,"40,646,186",12.20%
Urban 1,18.1%,17,0.06%,0.68%,"48,301,771",14.49%
Urban 2,3.8%,1,0.03%,0.46%,"9,721,138",2.92%
Urban 3,44.4%,163,0.23%,1.11%,"124,570,826",37.38%


In [47]:

data_all['R_CODE'] = 'X'
data_all.loc[data_all['R_CONTINUUM_CODE'].isin(['4','5','6','7','8','9']),'R_CODE'] = 'R'
data_all.loc[data_all['R_CONTINUUM_CODE'].isin(['1','2','3']),'R_CODE'] = 'U'

data_all.loc[(data_all['labels'] == 'Urban 1'), 'labels'] = 'Urban A'
data_all.loc[(data_all['labels'] == 'Urban 2'), 'labels'] = 'Urban A'
data_all.loc[(data_all['labels'] == 'Urban 3'), 'labels'] = 'Urban B'
print(silhouette_score(df_clustered_selected, data_all['labels']).round(2))

0.44


In [48]:
cluster_breakdown(data_all,['labels','R_CODE'])

GDP_2022_USD  Geo_FIPS FARM_PERSINC_2022_PCT  \
labels          R_CODE                                                
Rural Reframe 1 R              5.6%      1680                 5.73%   
                U              1.4%       385                 2.62%   
Rural Reframe 2 R              0.2%         8                 0.63%   
                U              8.9%       222                 0.41%   
Rural Reframe 3 R              0.1%         1                 3.11%   
                U              9.4%       112                 0.30%   
Rural Reframe 4 R              3.4%       269                 1.29%   
                U              4.7%       286                 0.65%   
Urban A         U             21.9%        18                 0.06%   
Urban B         U             44.4%       163                 0.23%   

                       IOC_I_AGR_22_PCT  TOT_POP_NUM TOT_POP_PCT  
labels          R_CODE                                            
Rural Reframe 1 R                 9.18%   27,050,693       8.12%  
                U                 4.38%    8,287,742       2.49%  
Rural Reframe 2 R                 2.31%    1,198,046       0.36%  
                U                 1.65%   38,264,971      11.48%  
Rural Reframe 3 R                 2.38%      255,956       0.08%  
                U                 1.35%   34,990,228      10.50%  
Rural Reframe 4 R                 3.44%   17,396,464       5.22%  
                U                 2.34%   23,249,722       6.98%  
Urban A         U                 0.67%   58,022,909      17.41%  
Urban B         U                 1.11%  124,570,826      37.38%

In [49]:

data_p = data_all.copy()


data_p.loc[(data_p['labels'] == 'Rural Reframe 4'), 'labels'] = 'Rural A'

data_p.loc[(data_p['labels'] == 'Rural Reframe 3'), 'labels'] = 'Urban Reframed'

data_p.loc[(data_p['labels'] == 'Rural Reframe 2'), 'labels'] = 'Urban Reframed'

#data_p.loc[(data_p['labels'] == 'Rural Reframe 1'), 'labels'] = 'Rural B'

data_p.loc[((data_p['labels'] == 'Rural Reframe 1') & (data_p['R_CODE'] == 'R')), 'labels'] = 'Rural B'
data_p.loc[((data_p['labels'] == 'Rural Reframe 1') & (data_p['R_CODE'] == 'U')), 'labels'] = 'Urban Reframed'

print(silhouette_score(df_clustered_selected, data_p['labels']).round(2))

0.26


In [55]:
cluster_breakdown(data_p,['labels'])

,GDP_2022_USD,Geo_FIPS,FARM_PERSINC_2022_PCT,IOC_I_AGR_22_PCT,TOT_POP_NUM,TOT_POP_PCT
labels,,,,,,
Rural A,8.1%,555,0.96%,2.87%,"40,646,186",12.20%
Rural B,5.6%,1680,5.73%,9.18%,"27,050,693",8.12%
Urban A,21.9%,18,0.06%,0.67%,"58,022,909",17.41%
Urban B,44.4%,163,0.23%,1.11%,"124,570,826",37.38%
Urban Reframed,20.0%,728,1.55%,3.06%,"82,996,943",24.90%


In [51]:
import datetime
import json
def saverun(run_name,meta_info,data):
    now = datetime.datetime.now(datetime.timezone(-datetime.timedelta(hours=5)))
    date_time = str(now.year) + str(now.month).zfill(2) + str(now.day).zfill(2) + "T" + str(now.hour).zfill(2) + str(now.minute).zfill(2) + str(now.second).zfill(2)

    meta_info['when'] = date_time
    filename = run_name + "_" + date_time+ ".csv"
    data.to_csv("/content/drive/MyDrive/FCB/Rural Reframe/Exports/Clusters/" + filename)

    skinny_data = data_p[['Geo_FIPS', 'R_CODE','cluster1','cluster2','labels']]
    skinny_data.to_csv("/content/drive/MyDrive/FCB/Rural Reframe/Exports/Clustering-" + filename)

    with open("/content/drive/MyDrive/FCB/Rural Reframe/Exports/Clusters/" +filename + "_Meta.json", "w") as outfile:
        json.dump(meta_info, outfile)

    print("Run Name: "+ run_name + "_" + date_time)

In [59]:
saverun('V25',meta_info,data_p)

Run Name: V25_20240709T134759


In [57]:
# prompt: generate a dataframe with only Geo_FIPS and labels




In [58]:
df_labels

,Geo_FIPS,R_CODE,cluster1,cluster2,labels
0,01001,U,1,3,Rural A
1,01003,U,1,2,Urban Reframed
2,01005,R,1,0,Rural B
3,01007,U,1,0,Urban Reframed
4,01009,U,1,3,Rural A
...,...,...,...,...,...
3139,56037,R,1,0,Rural B
3140,56039,R,1,0,Rural B
3141,56041,R,1,0,Rural B
3142,56043,R,1,0,Rural B
